In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# Desigualdad CHSH

*Estimación de uso: Dos minutos en un procesador Heron r2 (NOTA: Esta es solo una estimación. Su tiempo de ejecución puede variar.)*
## Antecedentes
En este tutorial, ejecutará un experimento en una computadora cuántica para demostrar la violación de la desigualdad CHSH con la primitiva Estimator.

La desigualdad CHSH, nombrada por los autores Clauser, Horne, Shimony y Holt, se utiliza para probar experimentalmente el teorema de Bell (1969). Este teorema afirma que las teorías de variables ocultas locales no pueden dar cuenta de algunas consecuencias del entrelazamiento en la mecánica cuántica. La violación de la desigualdad CHSH se utiliza para mostrar que la mecánica cuántica es incompatible con teorías de variables ocultas locales. Este es un experimento importante para comprender los fundamentos de la mecánica cuántica.

El Premio Nobel de Física 2022 fue otorgado a Alain Aspect, John Clauser y Anton Zeilinger en parte por su trabajo pionero en ciencia de la información cuántica y, en particular, por sus experimentos con fotones entrelazados que demostraron la violación de las desigualdades de Bell.
## Requisitos
Antes de comenzar este tutorial, asegúrese de tener instalado lo siguiente:

* Qiskit SDK v1.0 o posterior, con soporte de [visualización](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 o posterior
## Configuración

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Paso 1: Mapear entradas clásicas a un problema cuántico
Para este experimento, crearemos un par entrelazado en el cual mediremos cada qubit en dos bases diferentes. Etiquetaremos las bases para el primer qubit como $A$ y $a$ y las bases para el segundo qubit como $B$ y $b$. Esto nos permite calcular la cantidad CHSH $S_1$:

$$
S_1 = A(B-b) + a(B+b).
$$

Cada observable es $+1$ o $-1$. Claramente, uno de los términos $B\pm b$ debe ser $0$, y el otro debe ser $\pm 2$. Por lo tanto, $S_1 = \pm 2$. El valor promedio de $S_1$ debe satisfacer la desigualdad:

$$
|\langle S_1 \rangle|\leq 2.
$$

Expandir $S_1$ en términos de $A$, $a$, $B$ y $b$ resulta en:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Puede definir otra cantidad CHSH $S_2$:

$$
S_2 = A(B+b) - a(B-b),
$$

Esto lleva a otra desigualdad:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Si la mecánica cuántica puede ser descrita por teorías de variables ocultas locales, las desigualdades anteriores deben cumplirse. Sin embargo, como se demuestra en este tutorial, estas desigualdades pueden ser violadas en una computadora cuántica. Por lo tanto, la mecánica cuántica no es compatible con teorías de variables ocultas locales.
Si quiere aprender más teoría, explore [Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) con John Watrous.
Creará un par entrelazado entre dos qubits en una computadora cuántica creando el estado de Bell $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$. Usando la primitiva Estimator, puede obtener directamente los valores esperados necesarios ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$ y $\langle ab \rangle$) para calcular los valores esperados de las dos cantidades CHSH $\langle S_1\rangle$ y $\langle S_2\rangle$. Antes de la introducción de la primitiva Estimator, tendría que construir los valores esperados a partir de los resultados de medición.

Medirá el segundo qubit en las bases $Z$ y $X$. El primer qubit también se medirá en bases ortogonales, pero con un ángulo respecto al segundo qubit, que variaremos entre $0$ y $2\pi$. Como verá, la primitiva Estimator hace que ejecutar circuitos parametrizados sea muy fácil. En lugar de crear una serie de circuitos CHSH, solo necesita crear *un* circuito CHSH con un parámetro que especifique el ángulo de medición y una serie de valores de fase para el parámetro.

Finalmente, analizará los resultados y los graficará contra el ángulo de medición. Verá que para cierto rango de ángulos de medición, los valores esperados de las cantidades CHSH $|\langle S_1\rangle| > 2$ o $|\langle S_2\rangle| > 2$, lo que demuestra la violación de la desigualdad CHSH.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### Crear un circuito CHSH parametrizado
Primero, escribimos el circuito con el parámetro $\theta$, que llamamos `theta`. La [primitiva `Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) puede simplificar enormemente la construcción de circuitos y el análisis de salida proporcionando directamente valores esperados de observables. Muchos problemas de interés, especialmente para aplicaciones a corto plazo en sistemas ruidosos, pueden formularse en términos de valores esperados. La primitiva `Estimator` (V2) puede cambiar automáticamente la base de medición basándose en el observable proporcionado.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### Crear una lista de valores de fase para asignar más adelante
Después de crear el circuito CHSH parametrizado, creará una lista de valores de fase para asignar al circuito en el siguiente paso. Puede usar el siguiente código para crear una lista de 21 valores de fase que van de $0$ a $2 \pi$ con espaciado igual, es decir, $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Observables

Ahora necesitamos observables de los cuales calcular los valores esperados. En nuestro caso estamos mirando bases ortogonales para cada qubit, permitiendo que la rotación $Y$ parametrizada para el primer qubit varíe la base de medición casi continuamente con respecto a la base del segundo qubit. Por lo tanto, elegiremos los observables $ZZ$, $ZX$, $XZ$ y $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Paso 2: Optimizar el problema para la ejecución en hardware cuántico

Para reducir el tiempo total de ejecución del trabajo, las primitivas V2 solo aceptan circuitos y observables que se ajustan a las instrucciones y conectividad compatibles con el sistema objetivo (denominados circuitos y observables de arquitectura de conjunto de instrucciones (ISA)).

### Circuito ISA

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### Observables ISA

De manera similar, necesitamos transformar los observables para hacerlos compatibles con el backend antes de ejecutar trabajos con [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run). Podemos realizar la transformación usando el método `apply_layout` del objeto `SparsePauliOp`.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Paso 3: Ejecutar usando primitivas de Qiskit
Para ejecutar todo el experimento en una sola llamada a [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2).
Podemos crear una primitiva [`Estimator` de Qiskit Runtime](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2) para calcular nuestros valores esperados. El método `EstimatorV2.run()` toma un iterable de `bloques unificados primitivos (PUBs)`. Cada PUB es un iterable en el formato `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)